In [1]:
import os
import json
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import uuid
import subprocess
from multiprocessing import Pool
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from timeout_decorator import timeout, TimeoutError

In [2]:
inputFolder = "./input"

In [3]:
def trim(json, size=None, augment=3, likelihood=0.98):
    
    save="./faces/{0}.jpg"
    
    try:
        if json["likelihood"]<likelihood:
            return 
        img = cv2.imread(json["file"])
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        x = json["face"]["x"]
        y = json["face"]["y"]
        width = json["face"]["width"]
        height = json["face"]["height"]
        img = img[y-height//2:y+(height*3)//2, x-width//2:x+(width*3)//2,:]
        if size:
            img = cv2.resize(img,size)

        cv2.imwrite(save.format(str(uuid.uuid4())), img)
    except Exception as e:
        pass
        
@timeout(300)
def _create_dataset(filename):
    if (filename[-4::] in (".jpg",".png"))==False :
        return
    uid = str(uuid.uuid4())
    tmpFolder="tmp-" + uid
    os.makedirs(tmpFolder)
    shutil.copy(os.path.join(inputFolder, filename), os.path.join(tmpFolder, filename))
    cmd = "ruby /animeface-2009/animeface-ruby/proc_folder.rb {0} {0}/{1}.json".format(tmpFolder, uid)
    subprocess.run(cmd.split(" "))
    with open("{0}/{1}.json".format(tmpFolder, uid), "r") as fp:
        for line in fp:
            data=json.loads(line)
            trim(data, augment=None)
    
    shutil.rmtree(tmpFolder)

def create_dataset(filename):
    try:
        _create_dataset(filename)
    except TimeoutError:
        print("timeout:",filename)

In [4]:
with Pool() as p:
    imap = p.imap(create_dataset, os.listdir(inputFolder))
    list(tqdm(imap, total=len(os.listdir(inputFolder))))

timeout:
 dd39329b-2e45-4079-a0be-35315928a07d.jpgtimeout: 52473f87-ac02-49cb-8bc3-98936e60caf8.jpg
timeout: e2e4919d-0c40-44e1-991a-c9e0741fd81d.jpg
timeout: 7ddd8d03-ada6-4f8d-a3f4-e426159d9734.jpg
timeout: 1cce56e8-2174-473e-a03e-cdf8db5f74eb.jpg
timeout: d12dff89-2fdc-4ecb-809d-11689760dd40.jpg
timeout: 17363b1b-d0bc-4e0b-8594-f67b5ae8ba2a.jpg
timeout: a102a36f-a090-4d93-a1a2-709fc20994a5.jpg
timeout: 2e6a13b8-5a61-4780-8222-7a76b4933eb1.jpg
timeout: 9490a515-090e-4990-b5d9-18936598d6ed.jpg
timeout: ed538353-53fc-466d-81c9-8d5da2f2118b.jpg
timeout: ec6b96cf-6056-4273-9444-24271bf897c9.jpg
timeout: d6326c76-dabb-496e-9e20-1e38d8beacad.jpg
timeout: fe3bb605-520a-46e9-9f6b-34ed606e4a3e.jpg
timeout: 93d03304-d787-48b8-97fe-353feccef99b.jpg
timeout: 49c2c4ae-0639-47b2-8d09-f6bd6366adc1.jpg
timeout: 6dc177cd-f58c-4246-ba20-a113fbbd9a60.jpg
timeout: 51d5bc86-ec28-4c3f-8d91-78d17011247d.jpg
timeout: ffbd9787-a60f-4b61-ba0e-f36b0201ba83.jpg
timeout: 078a42e8-9d20-4fb8-8191-40f64629523a.jpg
